# Loading the dataset

In [ ]:
# Standard libraries
from typing import Tuple, Union
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

# Plotting libraries
from bokeh.models import DataRange1d, Range1d
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook
output_notebook(INLINE)

# Tables
from tabulate import tabulate

In [ ]:
# Set the path to the folder which contains the metrics files
path_metrics = "../river-blindness/Unet_Segmentation/modelEvaluationBackups/modelEvaluationWithoutDeepRed/logs/"
# Set line width for all diagrams
linewidth = 2

In [ ]:
# Functions for collecting and preparing the data for the graphs
def load_file(filename: str) -> Union[dict, list]:
    data_file = open(path_metrics + filename)
    data_json = json.load(data_file)
    data_file.close()

    return data_json


def means(data: dict) -> Tuple[list, list, list, list]:
    class0 = []
    class1 = []
    mean_cl0_cl1 = []
    x_vals = []   

    for key in data:
        class0.append(data[key][-1][0])
        class1.append(data[key][-1][1])
        mean_cl0_cl1.append((data[key][-1][0] + data[key][-1][1]) / 2)

    for y in range(1, len(class0) + 1):
        x_vals.append(y)
        
    return class0, class1, mean_cl0_cl1, x_vals


def means_per_epoch(filename: str) -> Tuple[list, list]:
    data_json = load_file(filename)
    mean_epochs_all_folds = []
    
    # Get x-axis values
    epochs_per_fold = settings['epochs_per_fold']
    eval_every = settings['eval_every']
    range_epochs = [*range(eval_every, epochs_per_fold + 1, eval_every)]  

    for fold in data_json:
        mean_epochs_one_fold = []
        
        for epoch in data_json[fold]:
            mean_epochs_one_fold.append((epoch[0] + epoch[1]) / 2)
        mean_epochs_all_folds.append(mean_epochs_one_fold)
        
    return range_epochs, mean_epochs_all_folds   


def train_val_values(filename: str) -> Tuple[list, list]:
    y_values = load_file(filename)
    x_values = []
    
    if type(y_values) == dict:
        y_values = list(y_values.values())
    
    for fold in y_values:
        x_length = len(fold)
        x_list = []
        for x in range(1, x_length + 1):
            x_list.append(x)
            
        x_values.append(x_list)
        
    return x_values, y_values
  

In [ ]:
# Functions for creating the tables
# Setting maximum length of table
chunk_size = 10

def check_table_length(x_values: list, chunks: list) -> list:
    
    if len(x_values) < chunk_size:
        
        return chunks[:len(x_values) + 1]
    
    return chunks


def creating_chunks(arr: list) -> list:
    offset = 0
    chunks = []
    
    while offset+chunk_size < len(arr):
        chunks.append(arr[offset:offset+chunk_size])
        offset += chunk_size
    chunks.append(arr[offset:offset+chunk_size])
    
    return chunks


def insert_header(chunks_x: list, chunks_y_0: list, chunks_y_1: list, index: str) -> Tuple[list, list, list]:
    
    for chunk in chunks_x:
        chunk.insert(0, index)
            
    for chunk in chunks_y_0:
        chunk.insert(0, 'Value Class 0')
        
    for chunk in chunks_y_1:
        chunk.insert(0, 'Value Class 1')
        
    return chunks_x, chunks_y_0, chunks_y_1


def table_two_values(x_values: list, y_values_0: list, y_values_1: list, index: str) -> list:
    chunks_x = creating_chunks(x_values)
    chunks_y_0 = creating_chunks(y_values_0)
    chunks_y_1 = creating_chunks(y_values_1)
    chunks_x, chunks_y_0, chunks_y_1 = insert_header(chunks_x, chunks_y_0, chunks_y_1, index)
    chunks = zip_chunks(chunks_x, chunks_y_0, chunks_y_1)
    chunks = check_table_length(x_values, chunks)
    
    return chunks


def zip_chunks(chunks_x: list, chunks_y_0: list, chunks_y_1: list) -> list:
    zips = []
    
    for i in range(len(chunks_x)):
        zips.append(list(zip(chunks_x[i], chunks_y_0[i], chunks_y_1[i])))
        
    while len(zips[-1]) < chunk_size:
        zips[-1].append(("",""))
        
    list_of_list_of_tuples = list(zip(*zips))
    zipped_chunks = []
    
    for list_of_tuples in list_of_list_of_tuples:
        z = []
        
        for _tuple in list_of_tuples:
            
            for _value in _tuple:
                z.append(_value)
        zipped_chunks.append(z)
        
    return zipped_chunks

# Creating Settings table
def settings_table(settings: dict) -> Tuple[list, str]:
    setting_keys = list(settings.keys())
    setting_values = list(settings.values())
    setting_descr = ''
    setting_keys.insert(0, '\033[1;3m' + 'Setting' + '\033[0m')
    setting_values.insert(0, '\033[1;3m' + 'Value' + '\033[0m')

    if (setting_keys[-1].lower() == 'description') :
        setting_descr = setting_values[-1]
        setting_keys = setting_keys[:-1]
        setting_values = setting_values[:-1]

    zips_tuples = list(zip(setting_keys, setting_values))  # Output: list with tuples
    zips_lists = list(map(list, zips_tuples))  # Output: list with lists

    return zips_lists, setting_descr

# Settings

In [ ]:
settings_file = open(path_metrics + 'settings.json')
settings = json.load(settings_file)
settings_file.close()

tabledata, description = settings_table(settings)

In [ ]:
print(tabulate(tabledata, headers='firstrow', tablefmt='fancy_grid', colalign=("left","left")))
print('\033[1;3m' + 'Description: ' + '\033[0m' + description)

# Metrics

## Dice Score - Means per Fold

In [ ]:
data_json = load_file('diceScoreMeans.json')
class0, class1, mean_cl0_cl1, x_vals = means(data_json)

In [ ]:
TOOLTIPS = [
    ("Fold", "@x"),
    ("Dice score", "@y"),
    ]
    
p = figure(
    x_axis_label = "Fold",
    y_axis_label = "Dice score",
    y_range=DataRange1d(range_padding = 0.1, follow = "end", only_visible = True),
    width=800,
    height=300, 
    tooltips=TOOLTIPS,
    title = "Dice score means",
    )

# Lines
p.line(x_vals, class0, legend_label = 'Class 0', color = 'blue', alpha=0.4, line_width=linewidth)
p.line(x_vals, class1, legend_label = 'Class 1', color = 'green', alpha=0.4, line_width=linewidth)

p.xaxis.minor_tick_line_color = None
p.xaxis.ticker = x_vals
p.yaxis.minor_tick_line_color = None

p.legend.click_policy="hide"
p.legend.location = "top_right"

show(p)

In [ ]:
table_data = table_two_values(x_vals, class0, class1, 'Fold')
print('\033[1;3m' + 'Mean Dice Scores per Fold' + '\033[0m')
print(tabulate(table_data, headers='firstrow', tablefmt='fancy_grid', colalign=("center","center")))

In [ ]:
print("Overall mean dice score of class 0: " + str(np.mean(class0)))
print("Overall mean dice score of class 1: " + str(np.mean(class1)))

## Dice Score - Means per Epoch

In [ ]:
range_epochs, mean_epochs_all_folds = means_per_epoch('diceScoreMeans.json')

In [ ]:
fold_count = 1

for fold in mean_epochs_all_folds:
    TOOLTIPS = [
    ("Epoch", "@x"),
    ("Dice score", "@y"),
    ]
    
    p = figure(
        x_axis_label = "Epoch",
        y_axis_label = "Dice score",
        y_range=DataRange1d(range_padding = 0.1, follow = "end", only_visible = True),
        width=800,
        height=300, 
        tooltips=TOOLTIPS,
        title = "Dice scores: Fold " + str(fold_count),
        )

    # Training losses
    p.line(range_epochs, fold, legend_label = 'Mean Dice Score', color = 'blue', alpha=0.4, line_width=linewidth)
    
    p.xaxis.minor_tick_line_color = None
    p.xaxis.ticker = range_epochs
    p.yaxis.minor_tick_line_color = None
    p.legend.click_policy="hide"
    p.legend.location = "bottom_left"

    show(p) 
    
    print("Mean Dice score of Fold " + str(fold_count) + ": " + str(np.mean(fold)))
    print("\n\n")
   
    fold_count = fold_count + 1

## Jaccard Index Means

In [ ]:
data_json = load_file('jaccardIndexMeans.json')
class0, class1, mean_cl0_cl1, x_vals = means(data_json)

In [ ]:
TOOLTIPS = [
    ("Fold", "@x"),
    ("Jaccard index", "@y"),
    ]
    
p = figure(
    x_axis_label = "Fold",
    y_axis_label = "Jaccard Index",
    y_range=DataRange1d(range_padding = 0.1, follow = "end", only_visible = True),
    width=800,
    height=300, 
    tooltips=TOOLTIPS,
    title = "Jaccard Index means",
    )

# Lines
p.line(x_vals, class0, legend_label = 'Class 0', color = 'blue', alpha=0.4, line_width=linewidth)
p.line(x_vals, class1, legend_label = 'Class 1', color = 'green', alpha=0.4, line_width=linewidth)

p.xaxis.minor_tick_line_color = None
p.xaxis.ticker = x_vals
p.yaxis.minor_tick_line_color = None

p.legend.click_policy="hide"
p.legend.location = "top_right"

show(p)

In [ ]:
table_data = table_two_values(x_vals, class0, class1, 'Fold')
print('\033[1;3m' + 'Mean Jaccard Index per Fold' + '\033[0m')
print(tabulate(table_data, headers='firstrow', tablefmt='fancy_grid', colalign=("center","center")))

In [ ]:
print("Overall mean Jaccard Index of class 0: " + str(np.mean(class0)))
print("Overall mean Jaccard Index of class 1: " + str(np.mean(class1)))

## Jaccard Indices per Fold

In [ ]:
range_epochs, mean_epochs_all_folds = means_per_epoch('jaccardIndexMeans.json')

In [ ]:
fold_count = 1

for fold in mean_epochs_all_folds:
    TOOLTIPS = [
    ("Epoch", "@x"),
    ("Jaccard Index", "@y"),
    ]
    
    p = figure(
        x_axis_label = "Epoch",
        y_axis_label = "Dice score",
        y_range=DataRange1d(range_padding = 0.1, follow = "end", only_visible = True),
        width=800,
        height=300, 
        tooltips=TOOLTIPS,
        title = "Dice scores: Fold " + str(fold_count),
        )

    # Training losses
    p.line(range_epochs, fold, legend_label = 'Mean Jaccard Index', color = 'blue', alpha=0.4, line_width=linewidth)
    
    p.xaxis.minor_tick_line_color = None
    p.xaxis.ticker = range_epochs
    p.yaxis.minor_tick_line_color = None
    p.legend.click_policy="hide"
    p.legend.location = "top_left"

    show(p) 
    
    print("Mean Dice score of Fold " + str(fold_count) + ": " + str(np.mean(fold)))
    print("\n\n")
   
    fold_count = fold_count + 1

## Train & Validation losses

In [ ]:
x_val, y_val = train_val_values('valLosses.json')
x_train, y_train = train_val_values('trainLosses.json')
fold_count = 1

for fold in range(0, len(y_val)):
    TOOLTIPS = [
    ("Epoch", "@x"),
    ("Loss", "@y"),
    ]
    
    p = figure(
        x_axis_label = "Epoch",
        y_axis_label = "Loss",
        y_range=DataRange1d(range_padding = 0.1, follow = "end", only_visible = True, start=0),
        width=800,
        height=300, 
        tooltips=TOOLTIPS,
        title = "Training and validation losses: Fold " + str(fold_count),
        )

    # Training losses
    p.line(x_train[fold], y_train[fold], legend_label = 'Training loss', color = 'blue', alpha=0.4, line_width=linewidth)
    # Validation losses
    p.line(x_val[fold], y_val[fold], legend_label = 'Validation loss', color = 'orange', alpha=0.7, line_width=linewidth)
    
    p.yaxis.minor_tick_line_color = None
    p.legend.click_policy="hide"

    show(p)
    
    print("Mean training loss: " + "\t" + str(np.mean(y_train[fold])))
    print("Mean validation loss: " + "\t" + str(np.mean(y_val[fold]))) 
    print("\n\n")
    
    fold_count = fold_count + 1